In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80","%EA%B3%B5%EC%A0%95%EC%9C%84","%EA%B3%B5%EC%A0%95%EB%AC%B8%EA%B5%AC"
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8","쉬즈블로그","%EC%89%AC%EC%A6%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "http://echoblog.net/images/sponsor-banner.png","%EB%AC%B4%EC%83%81%EC%A0%9C%EA%B3%B5",
                      "sponsor","banner","echoblog","reviewplace","www.storyn.kr","pick-me.kr"
                      , "%EC%A0%9C%ED%92%88%EC%A0%9C%EA%B3%B5"
                      , "reviewplace"
                      ,"site-tracker.08liter.com"
                      ,"08liter"]
# %EC%95%94%ED%96%89%EC%96%B4%EC%82%AC -> 암행어사...는 좀 일반명사 같아서 고민중
# %EB%B0%B0%EB%84%88 -> 배너 제외..

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 4
epochs = 10
IMG_HEIGHT = 120
IMG_WIDTH = 240


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_0728_v3(120_240)/cp-0010.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_0728_v3(120_240)/cp-0010.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [9]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
            with open("fake_img_url_list.txt", "a") as file:
                file.write(f'model_{last_index}_{score}.jpg\n')
                file.write(f'{img_url_final}\n')
                file.write("================================\n")
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            #blog.me, blog.naver 같은 형태일때만 digging!
            if(url_item.find("blog")<0):
                continue;
                
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");
            try:
                nickName = blogUrl.split('/')[-1].split('?')[0]
                postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]
            except:
                print("url parsing error : " , blogUrl);
                continue
            
            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

            for div_obj in blog_image_class:
#                 for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = div_obj.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")
                    #np array after save image(for crawling data)
                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                if(isSaveSuccess):  
                    index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [10]:
searchQuery = "골든몽키"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 가평 빠지 골든몽키 후기 , href : https://blog.naver.com/akfn3757?Redirect=Log&logNo=222043036370
----------------------------------
title : 가평빠지,춘천빠지 '골든몽키' 홍천강을 끼고있어서 물바운스 장난아냐~ , href : https://blog.naver.com/qkdcodnjs?Redirect=Log&logNo=222043509210
----------------------------------
title : 국내 여름휴가추천 물놀이 워터파크 골든몽키 , href : https://blog.naver.com/powertheone?Redirect=Log&logNo=222044230034
----------------------------------
title : 가평수상레져 가평빠지 골든몽키휴가~ , href : https://blog.naver.com/ksymd2000?Redirect=Log&logNo=222045310903
----------------------------------
title : 가평 가볼만한곳 | "골든몽키 수상레저 워터파크" :: 부지가 어마어마한 가평 빠지! , href : https://blog.naver.com/ns3452?Redirect=Log&logNo=222043588033
----------------------------------
title : 가평 빠지 최대규모의 물놀이 워터파크 골든몽키 , href : https://blog.naver.com/goldmoonbr?Redirect=Log&logNo=222030787737
----------------------------------
title : 가평 골든몽키 :: 쫄보 4명이 수상레저하러간  후기+꿀팁 , href : https://blog.naver.com/kk6074?Redirect=Log&logNo=222029904383
----------------

========pridiction========
predictions class :  [[0]]
predictions :  [[-1402.3513]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1245.6997]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1350.7023]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1058.048]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1439.4564]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1482.4785]]
========pridiction========
predictions class :  [[1]]
predictions :  [[605.70795]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-695.00836]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1813.4083]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-344.9135]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1365.9233]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1251.26

========pridiction========
predictions class :  [[0]]
predictions :  [[-231.52989]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1361.6903]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-808.1699]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1092.6003]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-645.8991]]
========pridiction========
predictions class :  [[1]]
predictions :  [[211.7353]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1320.4225]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1131.3969]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1425.9996]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1406.4254]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1543.3154]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1275.1781

========pridiction========
predictions class :  [[0]]
predictions :  [[-1589.3936]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2188.5247]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1193.3578]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1035.133]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-846.4793]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1227.6956]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1582.4824]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1400.6663]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-253.38722]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1292.3994]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1418.6721]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-952.29

========pridiction========
predictions class :  [[0]]
predictions :  [[-1302.0466]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-2585.8838]]
------ digging more -------
nickName :  joohr1838
postNumber :  222035508992
blogUrl :  https://m.blog.naver.com/joohr1838?Redirect=Log&logNo=222035508992
========pridiction========
predictions class :  [[0]]
predictions :  [[-1353.6134]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1382.6532]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-842.5274]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-356.33673]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1794.7778]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1061.775]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1526.2135]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-18

...

========pridiction========
predictions class :  [[0]]
predictions :  [[-1268.2003]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1652.7686]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1252.2048]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-487.90002]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1256.7301]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-648.4974]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1399.3252]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1631.8795]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1292.3252]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1653.4569]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-756.39624]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1152.

========pridiction========
predictions class :  [[0]]
predictions :  [[-424.97406]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1023.0203]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-689.1869]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1316.4214]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-457.95367]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1029.7241]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-764.8029]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-525.2349]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1332.9431]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-676.874]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1657.6958]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5.4702325]

title : [가평 빠지 추천] 골든몽키수상레저에서 빠지타임 *.* , href : https://lala1010.blog.me/221585455633
----------------------------------
title : 가평수상레저, 물놀이 워터파크 골든몽키! , href : https://blog.naver.com/bjkimbamtol?Redirect=Log&logNo=221326048076
----------------------------------
title : 가평 빠지 패키지 골든몽키 수상레저 워터파크 올여름은 여기서 불태워 버리쟈 , href : https://blog.naver.com/skysea_53?Redirect=Log&logNo=221319180633
----------------------------------
title : 2018 여름휴가 -가평 골든몽키수상레져 워터파크 / 플라워펜션 / 수동계곡 투어 , href : http://2motion.kr/221342991347
----------------------------------
title : 가평/춘천 골든몽키 수상레저워터파크 가평 빠지,가평 수상레저,가평 여행 가볼만한곳,국내 여름휴가추천 , href : https://blog.naver.com/hyacinthcmsh?Redirect=Log&logNo=221601302683
----------------------------------
title : 가평 수상레저 패키지 골든몽키 워터파크 : 가평 강촌 빠지 , href : https://blog.naver.com/kumiran89?Redirect=Log&logNo=221307309163
----------------------------------
title : 가평 수상레저 패키지 골든몽키 수상레저 워터파크 , href : https://blog.naver.com/yookjihye?Redirect=Log&logNo=221300965129
--------------

title : 가평수상레저 패키지 골든몽키 :) 여름 물놀이 꿀잼! , href : https://blog.naver.com/bkthdthd?Redirect=Log&logNo=221068916169
----------------------------------
title : [가평워터파크]강촌수상레저는 골든몽키에서, 신나게 즐겨요! , href : https://blog.naver.com/congsoon22?Redirect=Log&logNo=221068369126
----------------------------------
title : 발리우붓맛집- 우붓의 중국식당 골든 몽키 , href : https://4670ju.blog.me/220953480330
----------------------------------
title : 홀리카홀리카, 골든 몽키 글래머립 키트 , href : https://blog.naver.com/hmpark91?Redirect=Log&logNo=220524971182
----------------------------------
title : 빅토리(Victory Brewing) 골든 몽키(Golden Monkey) , href : https://blog.naver.com/sfoods?Redirect=Log&logNo=221149899013
----------------------------------
title : 홀리카 홀리카 골든몽키 글래머 립 3-step , href : https://blog.naver.com/7munbin?Redirect=Log&logNo=220510199067
----------------------------------
title : 홀리카홀리카 골든몽키로 매끈한입술 만들기 , href : https://blog.naver.com/wlgysnl23?Redirect=Log&logNo=220523631545
----------------------------------
title : 처음으로 홀리카 

------ digging more -------
nickName :  azzan
postNumber :  220523673630
blogUrl :  https://m.blog.naver.com/azzan?Redirect=Log&logNo=220523673630
------ digging more -------
nickName :  tjdal4992
postNumber :  220525288568
blogUrl :  https://m.blog.naver.com/tjdal4992?Redirect=Log&logNo=220525288568
------ digging more -------
nickName :  ppo1127
postNumber :  220555275197
blogUrl :  https://m.blog.naver.com/ppo1127?Redirect=Log&logNo=220555275197
------ digging more -------
nickName :  ny3535737
postNumber :  220284853947
blogUrl :  https://m.blog.naver.com/ny3535737?Redirect=Log&logNo=220284853947


In [11]:
# # blog gethering test
# def findBlogListTest(searchQuery, searchOption, page):
#     try:
#         print(f'\n###############{page}###############\n');
#         url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
#         html = requests.get(url)
#         # 1차, blog URL 찾기
#         soup = BeautifulSoup(html.text, 'html.parser')
#         index = 0;

#         url_list = [];
#         for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
#             child_item = li_item.find('a',{'class' : 'sh_blog_title'})
#             title = child_item.attrs['title']
#             href = child_item.attrs['href']

#             #url list 저장
#             url_list.append(href)

#             print(f'title : {title} , href : {href}')
#             print('----------------------------------')

#     except Exception as ex:
#         print("error catch : " , ex)
#         print("url : " ,img_url)

In [12]:
# searchQuery = "서울대입구역 하남돼지집"
# searchOption = 'sim'  #sim or date  
# page = 1
# end_page = 11
# # 1->11->21->순으로
# for i in list(range(page,end_page)):
#     if i>1:
#         page = page + 10;
#     findBlogListTest(searchQuery,searchOption,page)

In [13]:
# def search_function(tag):
#     return tag.attr('class') == 'se-imageStrip' or tag.attr('class') == 'se-image';

# #이미지 여러개 합쳐진 문제 
# url = 'https://m.blog.naver.com/danee_blog?Redirect=Log&logNo=221516896081'

# blog_html = requests.get(url)
# blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
# # blog_image_class = blog_soup.find_all('div',{'class' : 'se-image', 'class' : 'se-imageStrip'});
# blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

# for div_obj in blog_image_class:
#     img_url = div_obj.attrs['src'];
#     print(img_url)
# #     for idx, img_item in enumerate(div_obj.find_all('img')):
# #         img_url = img_item.attrs['src'];
# #         print(img_url)
# #         if(img_url.find("w80_blur") > 0):
# #             img_url = img_url.replace("w80_blur" , "w800")
#             #np array after save image(for crawling data)
# #         isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
#         #np arr from img_url for validation
#     #                     PredictionWithoutDownload(img_url)
# #         if(isSaveSuccess):  
# #             index = index+1
            
# # print("blog_image_class : " , blog_image_class.length)